# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import numpy as np
import psycopg2
import pandas as pd
from sql_queries import *

In [33]:
import sys
import json
sys.version, pd.__version__, np.__version__, psycopg2.__version__, json.__version__

('3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56) \n[GCC 7.3.0]',
 '1.1.5',
 '1.19.2',
 '2.8.6 (dt dec pq3 ext lo64)',
 '2.0.9')

In [31]:
!conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
absl-py                   0.10.0                   pypi_0    pypi
alabaster                 0.7.12                   pypi_0    pypi
apex                      0.1                      pypi_0    pypi
argon2-cffi               20.1.0                   pypi_0    pypi
ascii-graph               1.5.1                    pypi_0    pypi
async-generator           1.10                     pypi_0    pypi
attrs                     20.3.0             pyhd3eb1b0_0  
audioread                 2.1.9                    pypi_0    pypi
babel                     2.8.0                    pypi_0    pypi
backcall                  0.2.0                      py_0  
beautifulsoup4            4.9.3              pyhb0f4dca_0  
blas                      1.0                    openblas  
bleach                    3.2.1                    pypi_0    p

In [32]:
!python --version

Python 3.6.12 :: Anaconda, Inc.


In [2]:
conn = psycopg2.connect("host=local-postgres dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
dir_paths = get_files("./data/song_data")

In [5]:
filepath = dir_paths[0]

In [6]:
song_df = pd.read_json(filepath, orient='records', lines=True)
song_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = song_df.iloc[0][['song_id',
                             'title',
                             'artist_id',
                             'year',
                             'duration']].to_numpy()
song_data = [el.item() if type(el).__module__ == 'numpy' else el for el in song_data]
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
import math

In [10]:
artist_data = song_df.iloc[0][['artist_id', 
                               'artist_name', 
                               'artist_location', 
                               'artist_latitude', 
                               'artist_longitude']].to_numpy()
artist_data = [None if type(el).__module__ == 'numpy' and np.isnan(el) else el for el in artist_data]
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("./data/log_data")

In [13]:
filepath = log_files[0]
filepath

'/Docker/Developer/udacity/projects/data-modeling-with-postgres/data/log_data/2018/11/2018-11-27-events.json'

In [14]:
log_df = pd.read_json(filepath, orient='records', lines=True)
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
time_df = pd.DataFrame(data=pd.to_datetime(log_df.loc[log_df['page'] == 'NextSong']['ts'], unit='ms').
                       to_numpy()[:, None], columns=['start_time'])
time_df.head()

,start_time
0,2018-11-27 00:52:12.796
1,2018-11-27 00:56:49.796
2,2018-11-27 01:00:48.796
3,2018-11-27 01:33:16.796
4,2018-11-27 01:38:37.796


In [16]:
time_df = time_df.drop_duplicates(keep='first')

In [17]:
time_df['hour'] = time_df['start_time'].dt.hour
time_df['day'] = time_df['start_time'].dt.day
time_df['week'] = time_df['start_time'].dt.isocalendar().week
time_df['month'] = time_df['start_time'].dt.month
time_df['year'] = time_df['start_time'].dt.year
time_df['weekday'] = time_df['start_time'].dt.weekday

time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-27 00:52:12.796,0,27,48,11,2018,1
1,2018-11-27 00:56:49.796,0,27,48,11,2018,1
2,2018-11-27 01:00:48.796,1,27,48,11,2018,1
3,2018-11-27 01:33:16.796,1,27,48,11,2018,1
4,2018-11-27 01:38:37.796,1,27,48,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = pd.DataFrame(data=log_df[['userId', 'firstName', 'lastName', 'gender', 'level']].to_numpy(),
                       columns=['user_id', 'first_name', 'last_name', 'gender', 'level'])

user_df.head(), user_df.shape

(  user_id first_name  last_name gender level
 0      88   Mohammad  Rodriguez      M  paid
 1      88   Mohammad  Rodriguez      M  paid
 2      88   Mohammad  Rodriguez      M  paid
 3      38     Gianna      Jones      F  free
 4      38     Gianna      Jones      F  free,
 (303, 5))

In [20]:
user_df = user_df.drop_duplicates(keep='first')

user_df.shape

(27, 5)

In [21]:
user_df = user_df.loc[~pd.isnull(user_df).any(axis=1)]
                                
user_df.shape

(25, 5)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
for index, row in log_df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = (index, songid, artistid, row.userId, 
                     row.registration, row.ts, row.level, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.